In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing, ensemble

In [37]:
## load data
train = pd.read_csv('products.csv')
test = pd.read_csv('sampleSubmission.csv')

C:\Anaconda3\envs\aind-dog-copy\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
## fill missing values
#products['customerID'].fillna('BBID_0000', inplace=True)
train['promotion_description'].fillna('no_promo', inplace=True)
train['Gender'].fillna('no_gender', inplace=True)
train['State'].fillna('no_state', inplace=True)
train['PinCode'].fillna(-1, inplace=True)
train['DOB'].fillna("1", inplace=True)
#train['DOB'].dropna(inplace=True)

In [39]:
#handling missing data for product_code
train = train[np.isfinite(train['product_code'])]

In [40]:
cnt_srs = train['product_code'].astype('int64').value_counts().reset_index().head(20)
#cnt_srs = products['product_code'].value_counts().reset_index()
cnt_srs.columns = ['product_code', 'frequency_count']
cnt_srs

,product_code,frequency_count
0,300776411,60994
1,300776410,58437
2,108037568,51338
3,300785148,41500
4,108100382,35259
5,300785150,34794
6,300840018,31149
7,108100362,27963
8,108005676,27727
9,300111517,25360


In [41]:
prod_index = []
for i in cnt_srs['product_code']:
    prod_index.append(i)

In [42]:
product_joined = pd.merge(train, cnt_srs, on='product_code', how='inner')
products_less = product_joined[['customerID', 'Gender', 'State', 'product_code']].reset_index()
products_less.drop('index', axis=1, inplace=True)
products_to_join = pd.DataFrame(train['customerID']).reset_index()
products_to_join.drop('index', axis=1, inplace=True)

In [43]:
# Join the dummy variables to the main dataframe
train = pd.merge(products_to_join, products_less.drop_duplicates(subset=['customerID'], keep='last'), on='customerID', how='left')
train.head()

,customerID,Gender,State,product_code
0,BBID_2041,male,no_state,1.081e+08
1,BBID_2041,male,no_state,1.081e+08
2,BBID_2041,male,no_state,1.081e+08
3,BBID_2041,male,no_state,1.081e+08
4,BBID_2041,male,no_state,1.081e+08


In [44]:
train.dropna(inplace=True)

In [45]:
prod_dict = {'_'}

train = pd.get_dummies(train, columns=['product_code'], prefix=prod_dict)
train.columns=train.columns.str.replace('_','')
train.reset_index(drop=True).head()

,customerID,Gender,State,108000707.0,108005676.0,108037568.0,108100290.0,108100294.0,108100306.0,108100308.0,...,300070102.0,300111517.0,300481673.0,300776409.0,300776410.0,300776411.0,300781593.0,300785148.0,300785150.0,300840018.0
0,BBID_2041,male,no_state,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,BBID_2041,male,no_state,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,BBID_2041,male,no_state,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,BBID_2041,male,no_state,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,BBID_2041,male,no_state,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
train.columns

Index(['customerID', 'Gender', 'State', '108000707.0', '108005676.0',
       '108037568.0', '108100290.0', '108100294.0', '108100306.0',
       '108100308.0', '108100352.0', '108100362.0', '108100382.0',
       '300070102.0', '300111517.0', '300481673.0', '300776409.0',
       '300776410.0', '300776411.0', '300781593.0', '300785148.0',
       '300785150.0', '300840018.0'],
      dtype='object')

In [47]:
feature_cols = ["Gender","State"]

target_cols = ['108000707.0', '108005676.0',
               '108037568.0', '108100290.0', '108100294.0', '108100306.0',
               '108100308.0', '108100352.0', '108100362.0', '108100382.0',
               '300070102.0', '300111517.0', '300481673.0', '300776409.0',
               '300776410.0', '300776411.0', '300781593.0', '300785148.0',
               '300785150.0', '300840018.0']

In [69]:
train_size = 7476736
nrows = 1000000 # change this value to read more rows from train

start_index = train_size - nrows
for ind, col in enumerate(feature_cols):
    print(col)
    if train[col].dtype == "object":
        le = preprocessing.LabelEncoder()
        le.fit(list(train[col].values))
        temp_train_X = le.transform(list(train[col].values)).reshape(-1,1)[start_index:,:]
        #temp_test_X = le.transform(list(test[col].values)).reshape(-1,1)
    else:
        temp_train_X = np.array(train[col]).reshape(-1,1)[start_index:,:]
        #temp_test_X = np.array(test[col]).reshape(-1,1)
    if ind == 0:
        train_X = temp_train_X.copy()
        #test_X = temp_test_X.copy()
    else:
        train_X = np.hstack([train_X, temp_train_X])
        #test_X = np.hstack([test_X, temp_test_X])
    print(train_X.shape)


Gender
(1000000, 1)
State
(1000000, 2)


In [70]:
train_X

array([[ 1, 32],
       [ 1, 32],
       [ 1, 32],
       ..., 
       [ 2, 11],
       [ 2, 11],
       [ 2, 11]], dtype=int64)

In [72]:
train_y = train[['customerID', '108000707.0', '108005676.0',
               '108037568.0', '108100290.0', '108100294.0', '108100306.0',
               '108100308.0', '108100352.0', '108100362.0', '108100382.0',
               '300070102.0', '300111517.0', '300481673.0', '300776409.0',
               '300776410.0', '300776411.0', '300781593.0', '300785148.0',
               '300785150.0', '300840018.0']]

In [73]:
last_instance_df = train_y.drop_duplicates('customerID', keep='last')

In [75]:
train_y = np.array(train_y.fillna(0)).astype('str')[start_index:,1:]

AttributeError: 'numpy.ndarray' object has no attribute 'fillna'

In [76]:
print(train_X.shape, train_y.shape)

(1000000, 2) (1000000, 20)


In [77]:
print("Running Model..")
model = ensemble.RandomForestClassifier(n_estimators=10, max_depth=10, min_samples_leaf=10, n_jobs=4, random_state=2016)
model.fit(train_X, train_y)
#del train_X, train_y
#print("Predicting..")
#preds = np.array(model.predict_proba(test_X))[:,:,1].T

Running Model..


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=10,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=4, oob_score=False, random_state=2016,
            verbose=0, warm_start=False)

In [78]:
preds = np.array(model.predict_proba(train_X))[:,:,1].T

ValueError: could not broadcast input array from shape (1000000,2) into shape (1000000)